In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

import numpy as np
import pandas as pd
from copy import deepcopy
from Merge import merge
from copy import deepcopy
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from Battery import Battery
from Predictions import RF
from DPModel import DP_P2P

In [2]:
Start = '2022-06-19 00:00:00'
End = '2022-06-19 23:00:00'
N = len(pd.date_range(start=Start,end=End,freq="h"))

bat = Battery(max_capacity=13,max_charge=7)
max_number_states=20
houses = ["h16","h28","k28"]
merges = [merge(house) for house in houses]
x0 = (0,10,0)

rf_models = [RF(house) for house in houses]

stepsize = 2

In [3]:
num_loops = int(np.ceil(N/stepsize))
remainder = N%stepsize
length = stepsize
Start_i = Start
x0_i = x0

series = [pd.DataFrame() for _ in range(len(houses))]

for i in range(num_loops):
    if i == num_loops-1:
        length = length if remainder == 0 else remainder

    End_i = pd.date_range(start=Start_i, periods=24, freq="h")[-1]
    
    print(f"Loop number {i+1} of {num_loops}")
    print(f"Predicting period from {Start_i} to {End_i} (n = {len(pd.date_range(Start_i, End_i, freq='H'))})")
    
    preds_i = [rf.get_predictions(Start_i, End_i, get_carb = j==0) for j,rf in enumerate(rf_models)]
    
    for j in range(len(preds_i)-1):
        preds_i[j+1]["CO2Emission"] = preds_i[0]["CO2Emission"]
        
    print()
    print(f"Taking actions the first {length} actions from {Start_i} to {preds_i[0].index[length-1]}")
    
    dpp2p = DP_P2P(Start_i,End_i,preds_i,houses,deepcopy(bat))
    dpp2p.P2P_sol(x0_i, max_number_states=20, byday=True, verbose=True, acts=False)
    dpp2p.merges = merges
    dpp2p.ep = rf_models[0].get_emissions(Start_i, End_i)/1000
    dpp2p.constraints_ord = None
    dpp2p.P2P_sol(x0_i, max_number_states=20, byday=True, verbose=True, acts=True)
    
    series_i = dpp2p.all_series
    
    for j in range(len(series_i)):
        series_i[j] = series_i[j].iloc[:length, ]
    
    Start_i = pd.date_range(start=Start_i,periods=length+1,freq="h")[-1]
    
    temp = list(x0_i)
    for j in range(len(series_i)):
        temp[j] = series_i[j]["capacity_after"].iloc[length-1]
        
    x0_i = tuple(temp)
        
    for j in range(len(houses)):
        series[j] = pd.concat([series[j],series_i[j]])
    
    
    results_dict = {house: series[j] for j, house in enumerate(houses)}
    print()
    print("Done!")
    print()
    print()
    
for j in range(len(houses)):
    series[j]["cost_cummulative"] = series[j]["cost"].cumsum(axis=0)
    series[j]["emission_cummulative"] = series[j]["emission"].cumsum(axis=0)
    

Loop number 1 of 12
Predicting period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00 (n = 24)

Taking actions the first 2 actions from 2022-06-19 00:00:00 to 2022-06-19 01:00:00
The optimality constraints are as follows:
h16: 106.50824779287001, h28: 19.555665046679998, k28: 115.58960419764
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
The optimality constraints are as follows:
h16: 150.3193233877, h28: 69.99045636965, k28: 173.4388697353
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

Done!


Loop number 2 of 12
Predicting period from 2022-06-19 02:00:00 to 2022-06-20 01:00:00 (n = 24)

Taking actions the first 2 actions from 2022-06-19 02:00:00 to 2022-06-19 03:00:00
The optimality constraints are as follows:
h16: 104.51634349499, h28: 44.30694037418, k28: 125.2535354405
Period from 2022-06-19 02:00:00 to 2022-06-20 01:00:00
The optimality constraints are as follows:
h16: 154.531236957, h28: 96.55220834475, k28: 172.60081323560001
Period from 2022-06-19 02:00:00 to 

In [4]:
logic_series_print(series[0], p2p=True)

 hour     price    eprice   yield  surpl  grid  peer   ext   act    bef    deg    aft      cost   pcumsum      emis   ecumsum
    0:   1.5404,   0.1370,   -5.5,  -5.5,  5.5,  0.0,  0.0,  0.0,   0.0,   0.0,   0.0,  13.9721,  13.9721,   0.7535,   0.7535
    1:   1.4134,   0.1020,   -5.3,  -5.3,  5.3,  0.0,  0.0,  0.0,   0.0,   0.0,   0.0,  12.7911,  26.7632,   0.5406,   1.2941
    2:   1.2313,   0.1460,   -4.2,  -4.2,  4.2,  0.0,  0.0,  0.0,   0.0,   0.0,   0.0,   9.3715,  36.1347,   0.6132,   1.9073
    3:   1.0400,   0.1550,   -3.8,  -3.8,  3.8,  0.0,  0.0,  0.0,   0.0,   0.0,   0.0,   7.7519,  43.8866,   0.5890,   2.4963
    4:   0.8971,   0.1550,   -3.8,  -3.8,  3.8,  0.0,  0.0,  0.0,   0.0,   0.0,   0.0,   7.2091,  51.0957,   0.5890,   3.0853
    5:   0.8254,   0.1850,   -3.4,  -9.4,  9.4,  0.0,  6.0,  6.0,   0.0,   0.0,   6.0,  17.1590,  68.2547,   1.7390,   4.8243
    6:   0.8373,   0.2410,   -3.5,   2.5, -0.0, -2.5, -2.5, -6.0,   6.0,   6.0,   0.0,  -2.0932,  66.1616,  -0.6025,  

In [5]:
logic_series_print(series[1], p2p=True)

 hour     price    eprice   yield  surpl  grid  peer   ext   act    bef    deg    aft      cost   pcumsum      emis   ecumsum
    0:   1.5404,   0.1370,   -4.8,   2.2, -0.0, -2.2, -2.2, -7.0,  10.0,  10.0,   3.0,  -3.3889,  -3.3889,  -0.3014,  -0.3014
    1:   1.4134,   0.1020,   -3.3,  -0.3,  0.3,  0.0,  0.0, -3.0,   3.0,   3.0,   0.0,   0.7240,  -2.6648,   0.0306,  -0.2708
    2:   1.2313,   0.1460,   -3.3,  -3.3,  3.3,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   7.3633,   4.6985,   0.4818,   0.2110
    3:   1.0400,   0.1550,   -2.8,  -2.8,  2.8,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   5.7119,  10.4104,   0.4340,   0.6450
    4:   0.8971,   0.1550,   -3.0,  -3.0,  3.0,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   5.6914,  16.1018,   0.4650,   1.1100
    5:   0.8254,   0.1850,   -2.7,  -3.4,  3.4,  0.0,  0.7,  0.7,   0.0,   0.0,   0.7,   6.2065,  22.3083,   0.6290,   1.7390
    6:   0.8373,   0.2410,   -2.6,  -1.9,  1.9,  0.0,  0.0, -0.7,   0.7,   0.7,   0.0,   3.4908,  25.7990,   0.4579,  

In [6]:
logic_series_print(series[2], p2p=True)

 hour     price    eprice   yield  surpl  grid  peer   ext   act    bef    deg    aft      cost   pcumsum      emis   ecumsum
    0:   1.5404,   0.1370,   -7.3,  -7.3,  5.1,  2.2,  0.0, -0.0,   0.0,   0.0,   0.0,  16.3448,  16.3448,   0.6987,   0.6987
    1:   1.4134,   0.1020,   -6.3,  -6.3,  6.3,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,  15.2045,  31.5493,   0.6426,   1.3413
    2:   1.2313,   0.1460,   -5.6,  -5.6,  5.6,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,  12.4953,  44.0446,   0.8176,   2.1589
    3:   1.0400,   0.1550,   -4.1,  -4.1,  4.1,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   8.3639,  52.4085,   0.6355,   2.7944
    4:   0.8971,   0.1550,   -4.1,  -4.1,  4.1,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   7.7783,  60.1868,   0.6355,   3.4299
    5:   0.8254,   0.1850,   -3.5,  -3.5,  3.5,  0.0,  0.0, -0.0,   0.0,   0.0,   0.0,   6.3890,  66.5758,   0.6475,   4.0774
    6:   0.8373,   0.2410,   -3.2,  -3.2,  0.7,  2.5,  0.0, -0.0,   0.0,   0.0,   0.0,   3.3792,  69.9550,   0.1687,  